In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow import keras
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip -q "/content/drive/MyDrive/train_data.zip"

In [5]:
img_gen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    validation_split=0.2
)

In [6]:
BATCH_SIZE = 32
train_ds = img_gen.flow_from_directory(directory = '/content/train_data' , batch_size = BATCH_SIZE ,
                                       shuffle = True , class_mode = 'sparse' ,
                                       target_size = (299,299) , subset = 'training' , seed = 42)
val_ds = img_gen.flow_from_directory(directory = '/content/train_data' , batch_size = BATCH_SIZE ,
                                     shuffle = False  , class_mode = 'sparse' ,
                                     target_size = (299,299) , subset = 'validation', seed = 42)

Found 7994 images belonging to 10 classes.
Found 1996 images belonging to 10 classes.


In [7]:
from keras.src.saving.saving_lib import optimizer
from keras.applications.inception_resnet_v2 import InceptionResNetV2

num_classes = 10
base_model_1 = InceptionResNetV2(weights='imagenet' , include_top = False)
x = base_model_1.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1024, activation='relu')(x)
predictions = keras.layers.Dense(num_classes, activation='softmax')(x)
model_1 = keras.Model(inputs=base_model_1.input, outputs=predictions)
model_1.compile(loss = 'sparse_categorical_crossentropy' ,
                     optimizer = 'adam' , metrics = ['accuracy'])
for layer in model_1.layers[:775] :
  layer.trainable = False
for layer in model_1.layers[775:] :
  layer.trainable = True
check_inc_res = keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/inc_res_v2_1.h5', monitor='val_accuracy', save_best_only=False)

219055592/219055592 [==============================] - 7s 0us/step


In [ ]:
model_1.fit(train_ds , validation_data = val_ds , epochs = 10 , callbacks=[check_inc_res] )

Epoch 1/10
250/250 [==============================] - ETA: 0s - loss: 1.9500 - accuracy: 0.3174

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


250/250 [==============================] - 405s 1s/step - loss: 1.9500 - accuracy: 0.3174 - val_loss: 2.6359 - val_accuracy: 0.1959
Epoch 2/10
250/250 [==============================] - 297s 1s/step - loss: 1.6596 - accuracy: 0.4234 - val_loss: 1.8224 - val_accuracy: 0.3938
Epoch 3/10
 39/250 [===>..........................] - ETA: 3:40 - loss: 1.5642 - accuracy: 0.4573